https://discuss.python.org/t/sorting-overlap-date-and-time/16820?u=vbrozik

In [44]:
from dataclasses import dataclass, field
from typing import NamedTuple
from types import SimpleNamespace


TERMINAL_WIDTH = 100   # text terminal width in characters
TERMINAL_SPACE = 15    # space to keep on the side of a graph


class NumInterval(NamedTuple):
    """Numeric interval."""
    start: float
    end: float
    
    def __add__(self, amount: float) -> NumInterval:
        """Define NumInterval + float as interval shift operation."""
        return NumInterval(self.start + amount, self.end + amount)

    def __mul__(self, amount: float) -> NumInterval:
        """Define NumInterval * float as interval scale operation."""
        return NumInterval(self.start * amount, self.end * amount)
    
    def length(self) -> float:
        """Return interval length."""
        return self.end - self.start

    
@dataclass
class AllocationInterval:
    """Resource allocation time interval."""
    name: str
    interval: NumInterval


@dataclass
class IntervalTransformer:
    """Transfor interval to different range or textual representation."""
    input_range: NumInterval
    output_max: float = TERMINAL_WIDTH - TERMINAL_SPACE
    output_characters: SimpleNamespace = field(default_factory=lambda: SimpleNamespace(
            inside = '#',
            outside = '.',
            ex_left = '<',
            ex_right = '>',
            split = '|',
            ))
    
    def __post_init__(self) -> None:
        self._shift = - self.input_range.start
        self._scale = self.output_max / self.input_range.length()
    
    def transform(self, input: NumInterval) -> NumInterval:
        """Transform the interval to the output range."""
        return (input + self._shift) * self._scale
    
    def string(self, input: NumInterval) -> str:
        """Transform the interval to its textual representation."""
        transformed = self.transform(input)
        return (
                self.output_characters.outside * int(transformed.start)
                + self.output_characters.inside * int(transformed.end - transformed.start)
                + self.output_characters.outside * int(self.output_max - transformed.end))

In [40]:
i1 = NumInterval(20, 60)
i1

NumInterval(start=20, end=60)

In [51]:
input_range = NumInterval(0, 100)
it = IntervalTransformer(input_range)
# it.transform(i1)
it.string(i1)

'.................##################################..................................'

In [50]:
import random

interval_max = TERMINAL_WIDTH - TERMINAL_SPACE
range_all = NumInterval(0, interval_max)
transformer = IntervalTransformer(range_all, interval_max)

random_intervals = [
    NumInterval(*sorted((random.uniform(0, interval_max), random.uniform(0, interval_max))))
    for _ in range(30)]
for interval in random_intervals:
    print(transformer.string(interval))

.................................................########...........................
.....................##############################################................
....................############################....................................
......................#######.......................................................
..##############################################################....................
..............####################################################..................
..............................#########################.............................
#########################...........................................................
....####################################################################...........
...................................................................##...............
..................................#############################....................
....................................................................